# 順序ロジットモデル

前節までのロジットモデルやプロビットモデルでは被説明変数が2つの値から1つを選ぶケースを扱っていました。ここでは、被説明変数が3つ以上の値から一つを選ぶ離散選択モデルを扱います。具体的には、順序ロジットモデル（orderd logit model、本頁）と多項ロジットモデル（multinomial logit model、次頁）を取り上げます。

**順序ロジットモデル（ordered logt model）**
順序ロジットモデルは、被説明変数が取りうる値が3つ以上、かつ、それらに何らかの順序がある場合に適用します。


```{note}
順序ロジスティック回帰で扱う例としては、「優れている、どちらとも言えない、劣っている」の中から1つを選ぶ場合や、公共交通機関を利用する頻度が「ほぼ毎日、週5日程度、週２-3日程度、週1日程度、ほぼない」から1つを選ぶ場合などが該当します。

<b>例1</b>：ある研究で、個々人がスマートフォンで位置情報を取得されることについてどのような意識を持っているかを調べています。個々人位置情報が収集されることに抵抗があるかどうか「とても当てはまる」、「どちらかというと当てはまる」、「どちらでもない」、「どちらかというと当てはまらない」、「全く当てはまらない」という5つの選択肢から選んでもらうことにしました。どのような属性（学歴・就業状況・年齢など）を持つ人ほど、その意識に当てはまりやすいのかを推定します。ここで、分析に当たっては5つの選択肢の間の「距離」が等しくないと考えます。
```

-----

ここでは、UCLAのページ(https://stats.oarc.ucla.edu/r/dae/ordinal-logistic-regression/)で公開されている順序ロジスティック回帰の説明を参考にPythonで分析をしてみましょう。

分析例として扱うのは、大学院への進学の意思決定与える影響要因の調査です。大学3年生に、大学院に出願する可能性は低いか、やや高いか、非常に高いかを尋ねたサンプルデータを用います。この例では、被説明変数変数には3つのカテゴリーがあり、説明変数としては親の教育状況、学部が公立か私立か、現在のGPAに関するデータなどがあります。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from statsmodels.miscmodels.ordinal_model import OrderedModel

%matplotlib inline

まずはデータをダウンロードして最初の３行を表示させてみましょう。

In [ ]:
url = "https://stats.idre.ucla.edu/stat/data/ologit.dta"
data_student = pd.read_stata(url)

In [ ]:
data_student.head(3)

各行に調査に回答した大学生のデータが収録されています。`apply`列に、大学院への進学について「可能性が低い」「やや高い」「非常に高い」という回答結果が入っています。また、以下の変数が確認できます。
* `pared`: 少なくとも片方の親が大学院の学位を持っているかどうかを示す0/1の変数
* `public`: 学部が公立であれば1、私立であれば0とするダミー変数
* `gpa`: 学生の成績平均で0から4の間の値をとる

以下、`pared`, `public`, `gpa`を説明変数、`apply`を被説明変数とする順序ロジットモデルを考えます。

In [ ]:
data_student.dtypes

`pared`が`category`というタイプとなっていることがわかります。

```{seealso}
`pandas`でのカテゴリ変数の扱いについては次のページを確認してください。
https://pandas.pydata.org/docs/user_guide/categorical.html
```

被説明変数`apply`の3つのカテゴリごとにGPAの値が`gpa`どのように分布しているのかを確認します

In [ ]:
fontsize = 10
fig, ax = plt.subplots(figsize=(5,3))
target_cat = data_student['apply'].unique()
data = [data_student.loc[data_student['apply'].isin([x]),'gpa'] for x in target_cat]
ax.boxplot(data, 
            labels = target_cat,
            meanprops = {"marker": "^",
             "markerfacecolor": "red",
             "markeredgecolor": "red",
             "markersize": "8"},
            showmeans=True)
ax.set_title('Box plot of average GPA', fontsize=fontsize)
ax.set_xticklabels(ax.get_xticklabels(), fontsize=fontsize)
ax.grid()
plt.show()

ヒストグラムでも同じデータを可視化で確認してみます

In [ ]:
fontsize = 10
fig, ax = plt.subplots(figsize=(5,3))
ax.hist(data, label=target_cat, alpha = 0.9, color = ['#fdae61','#abdda4','#2b83ba'])
ax.legend(frameon=False, fontsize=fontsize)
ax.set_title("Histogram of average GPA", fontsize=fontsize)
plt.show()

少なくとも片方の親が大学院の学位を持っているかどうか`pared`と学部が公立であるかどうか`public`によって、大学院への進学意思はどのように変わるのかも平均値と標準偏差から概観します・

In [ ]:
data_student.groupby(['apply'])[['public','pared']].agg(['mean', 'std'])

$Y_i^*$は潜在変数で、観測された順序変数$Y$が何に等しいかを決定します。潜在変数$Y^*$はいくつかの閾値$m_j$を持っていて、観測された変数$Y$の値は、特定の閾値を超えたかどうかに依存します。例えば、$j=5$では、順序ロジットモデルは以下のような式となります。

 $$Y_i = \begin{cases}
        5 & \text{if} & m_5<Y_i^*\\
        4 & \text{if} & m_4<Y_i^*\leq m_5\\
        3 & \text{if} & m_3<Y_i^*\leq m_4 & \text{where } & Y^*_i=a+bX_i+u_i\\
        2 & \text{if} & m_2<Y_i^*\leq m_3\\
        1 & \text{if} & Y_i^* \leq m_2\\
        \end{cases}$$

$m_j$はカットオフ（cut-off）と呼ばれ、m_jより大きいか小さかでYが特定の値を取る確率を求めます。

`statsmodels`を用いて推定します。
 

In [ ]:
mod_logit = OrderedModel(data_student['apply'],
                        data_student[['pared', 'public', 'gpa']],
                        distr='logit')

res_logit = mod_logit.fit(method='bfgs')
print(res_logit.summary())

上の結果で「unlikely/somewhat likely」と「somewhat likely/very likely」はカットポイントと呼ばれる切片で、3つのグループを生成するために潜在変数がどこで切断されるかを示しています。一般的にこれらの結果は結果の解釈に用いません。

オッズ比も求めます。

In [ ]:
params = res_logit.params
conf = res_logit.conf_int()
conf['Odds Ratio'] = params
conf.columns = ['2.5%', '97.5%', 'Odds Ratio']
print(np.exp(conf))

Formulaを書く方法でもOKです。

In [ ]:
modf_logit = OrderedModel.from_formula("apply ~ 1 + pared + public + gpa", data_student,
                                      distr='logit')
resf_logit = modf_logit.fit(method='bfgs')
print(resf_logit.summary())